In [181]:
from datetime import datetime, timedelta
import pandas as pd 
import datetime as dt
import fnmatch
import os
import pymysql
from sqlalchemy import create_engine
import sys
import json
import numpy as np
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

# Obtener la fecha de hoy
hoy = datetime.now()

# Calcular la fecha de ayer
ayer = hoy - timedelta(days=1)

# Formatear la fecha como "aaaa-mm-dd"
fecha_ayer_formato = ayer.strftime("%Y-%m-%d")

fecha_ayer_formato

'2024-04-22'

In [182]:
def encontrar_registros_adicionales(df1, df2, columnas_comparar):
    """
    Encuentra los registros adicionales en el segundo DataFrame en comparación con el primero.

    Parameters:
    - df1: Primer DataFrame
    - df2: Segundo DataFrame
    - columnas_comparar: Lista de columnas en las cuales comparar los DataFrames

    Returns:
    - DataFrame con los registros adicionales en el segundo DataFrame
    """
    # Realiza un merge con indicator=True
    merged = pd.merge(df1, df2, on=columnas_comparar, how='outer', indicator=True)

    # Filtra los resultados para obtener solo los registros en el segundo DataFrame
    adicionales_df2 = merged.loc[lambda x: x['_merge'] == 'right_only']

    # Elimina la columna _merge utilizada solo para la comparación
    adicionales_df2 = adicionales_df2.drop('_merge', axis=1)

    return adicionales_df2


In [183]:
cnx = pymysql.connect(host='159.89.90.197', port=3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                           charset='utf8',db = 'volana')

In [184]:
path = f'./JERS/data/'
if not os.path.exists(path + fecha_ayer_formato): 
    os.makedirs(path+ fecha_ayer_formato) 

**Obtener fecha más actual**

In [185]:
from datetime import datetime

# Convierte las fechas a objetos datetime
fechas_dt = [datetime.strptime(fecha, '%Y-%m-%d') for fecha in os.listdir("./JERS/data/") if fecha != fecha_ayer_formato]

# Encuentra la fecha más reciente
ultima_fecha = max(fechas_dt)

ultima_fecha = ultima_fecha.strftime('%Y-%m-%d')

In [186]:
ultima_fecha

'2024-04-21'

# DEU

In [187]:
cursor = cnx.cursor()
sql_query = f"""
            SELECT 
ids.id_distribuidor as `Clave Referencia`,
if(ids.tipo_prestamo = 'PRESTAMO PERSONAL RED', 'RED', cd.tipo_distribuidor) as 'tipo distribuidor',
"6" as `Tipo Operación`,
ids.id_red as `Contratante`,
IFNULL(dd.apellido_1,"X") as `Paterno Deudor`,
IFNULL(dd.apellido_2,"X") as `Materno Contratante`,
CONCAT(ifnull(dd.nombre_1,"")," ",ifnull(dd.nombre_2,"")) as `Nombre Contratante`,
"PF" as `Tipo de Persona`,
ifnull(dd.rfc,"X") as `RFC`,
CONCAT(ifnull(dd.calle_y_numero,"")," ",ifnull(dd.colonia,"")) as `Dirección`,
IFNULL(dd.delegacion,"X") as `Ciudad`,
IFNULL(dd.cp,"X") as `CP`,
IFNULL(dd.estado,"X") as `Estado`,
"N/A" as `Email`,
IFNULL(dd.telefono,"X") as `Teléfono`,
date(cd.fecha_aut_linea) as `Fecha Firma Contrato`,
if(cd.tipo_distribuidor = 'RED',ids.fin_credito,'2099-12-31') as `Fecha Vencimiento Contrato`,
if(cd.tipo_distribuidor = 'RED','16','0') as `Plazo`,
ids.id_red as `Crédito`,
"N" as `Anexo`,
cd.linea_de_credito  as `Monto Contrato`,
if(cd.tipo_distribuidor = 'RED','Semanal','Quincenal') as `Frecuencia de Pago`,
cd.proximo_pago_total_pago as `Pago Periódico`,
cd.linea_de_credito as `Monto Pagare`,
if(cd.tipo_distribuidor = 'RED',ids.tasa,"") AS `Tasa`,
dd.fecha_nacimiento as `Fecha de nacimiento contratante`,
IFNULL(dd.sexo,"X") as `Sexo`,
IFNULL(dd.curp,"X") as `CURP`
FROM cartera_distribuidor cd
LEFT JOIN (
		select 
			ca.id_distribuidor, 
            max(ca.id_red) as 'id_red', 
            max(ca.tipo_prestamo) as 'tipo_prestamo',
            max(tasa) as 'tasa',
            max(fin_credito) as 'fin_credito'
			from (
				select 
				if(tipo_prestamo = 'PRESTAMO PERSONAL RED', id_distribuidor_linea_de_credito, id_distribuidor) as 'id_distribuidor', 
				id_distribuidor as 'id_red',
				tipo_prestamo,
                tasa,
                fin_credito
				from colocacion_asociado
			) as ca 
		group by ca.id_distribuidor
) ids on cd.id_distribuidor = ids.id_distribuidor
LEFT JOIN demograficos_distribuidores dd on ids.id_distribuidor = dd.id
WHERE fecha = '{fecha_ayer_formato}' and 
ids.id_distribuidor IS NOT NULL and
dd.nombre_1 IS NOT NULL;
            """
cursor.execute(sql_query)
DEU = cursor.fetchall()

In [188]:
DEU = pd.DataFrame(DEU,columns = ['Clave Referencia', 'tipo distribuidor', 'Tipo Operación',
       'Contratante', 'Paterno Deudor', 'Materno Contratante',
       'Nombre Contratante', 'Tipo de Persona', 'RFC', 'Dirección', 'Ciudad',
       'CP', 'Estado', 'Email', 'Teléfono', 'Fecha Firma Contrato',
       'Fecha Vencimiento Contrato', 'Plazo', 'Crédito', 'Anexo',
       'Monto Contrato', 'Frecuencia de Pago', 'Pago Periódico',
       'Monto Pagare', 'Tasa', 'Fecha de nacimiento contratante', 'Sexo',
       'CURP'])

In [189]:
DEU.head()

,Clave Referencia,tipo distribuidor,Tipo Operación,Contratante,Paterno Deudor,Materno Contratante,Nombre Contratante,Tipo de Persona,RFC,Dirección,...,Crédito,Anexo,Monto Contrato,Frecuencia de Pago,Pago Periódico,Monto Pagare,Tasa,Fecha de nacimiento contratante,Sexo,CURP
0,1,CLUB,6,1,DIAZ,FLORES,MARIA DE LOS ANGELES,PF,DIFA770122,SAN BERNARDINO no ext.14 Cofradia de la Luz,...,1,N,62000,Quincenal,0.0,62000,,1977-01-22,FEMENINO,DIFA770102MJCZLN08
1,100002,RED,6,139856,PALMA,FIGUEIRAS,EVA,PF,PAFE851219,NTE 5 A no ext. 36 Orizaba Centro,...,139856,N,10000,Semanal,0.0,10000,178.63999938964844,1985-12-19,FEMENINO,PAFE851219MVZLGV09
2,100011,RED,6,101435,HERNANDEZ,BOLAINA,SHEILA MARIA,PF,HEBS000728,BAJIO2A SECC no ext.SN EL BAJIO COL,...,101435,N,5000,Semanal,0.0,5000,110,2000-07-28,FEMENINO,HRBLSH00072827M100
3,100016,RED,6,101435,HERNANDEZ,BOLAINA,TRINIDAD DEL CARMEN,PF,HEBT940712,CALLE 4 SN JUNTO AL KINDER no ext.SN Bajío ...,...,101435,N,5000,Semanal,0.0,5000,110,1994-07-12,FEMENINO,HEBT940712MTCRLR01
4,100025,RED,6,142629,ORTEGA,GALINDO,NANCY,PF,OEGN850723,CEDROS no ext.42 San Salvador Atenco,...,142629,N,2000,Semanal,0.0,2000,140,1985-07-23,FEMENINO,OEGN850723MDFRLN00


## Adecuaciones

In [190]:
DEU_id = np.array(DEU['Contratante'].unique(), dtype = int)
DEU['Fecha Firma Contrato'] = pd.to_datetime(DEU['Fecha Firma Contrato'])
print(DEU.shape)
for name in DEU.select_dtypes(include=['object']):
    DEU[name] = DEU[name].str.replace("|", " ")
print("Hay que revisar DEU" if (DEU.isna().any(0).sum() > 2) else "Todo bien con DEU")
DEU.head(1)

(10909, 28)
Todo bien con DEU


,Clave Referencia,tipo distribuidor,Tipo Operación,Contratante,Paterno Deudor,Materno Contratante,Nombre Contratante,Tipo de Persona,RFC,Dirección,...,Crédito,Anexo,Monto Contrato,Frecuencia de Pago,Pago Periódico,Monto Pagare,Tasa,Fecha de nacimiento contratante,Sexo,CURP
0,1,CLUB,6,1,DIAZ,FLORES,MARIA DE LOS ANGELES,PF,DIFA770122,SAN BERNARDINO no ext.14 Cofradia de la Luz,...,1,N,62000,Quincenal,0.0,62000,,1977-01-22,FEMENINO,DIFA770102MJCZLN08


In [191]:
DEU.to_csv(f'{path}{fecha_ayer_formato}/DEU_{fecha_ayer_formato}.txt', sep ='|', index = False, header = False)

In [192]:
DEU_ant = pd.read_csv(f"{path}{ultima_fecha}/DEU_{ultima_fecha}.txt", sep = '|', header = None)
DEU_ant.columns = DEU.columns
DEU_ant.shape[0], DEU.shape[0]

(10909, 10909)

In [193]:
DEU = pd.read_csv(f"{path}{fecha_ayer_formato}/DEU_{fecha_ayer_formato}.txt", sep = '|', header = None)
DEU.columns = DEU_ant.columns

In [194]:
DEU.Email = DEU.Email.fillna("")
DEU_ant.Email = DEU_ant.Email.fillna("")

In [195]:
np.setdiff1d(DEU['Contratante'], DEU_ant['Contratante'])

array([183377, 185721], dtype=int64)

In [196]:
DEU_new = DEU[DEU['Contratante'].isin(np.setdiff1d(DEU['Contratante'], DEU_ant['Contratante']))]

DEU_new = DEU_new.drop_duplicates(subset = ["Clave Referencia"])

#Los para enviarse
DEU_new.to_csv(f'{path}/{fecha_ayer_formato}/DEU_{fecha_ayer_formato}_tesoreria.csv', index = False)

DEU_new.to_csv(f'{path}{fecha_ayer_formato}/DEU_{fecha_ayer_formato}_diferencial.txt', sep ='|', index = False, header = False)

In [197]:
DEU_new.shape

(3, 28)

# MOVA

In [198]:
cursor = cnx.cursor()
sql_query = f"""
select 
    id_distribuidor as `Contrato`,
    "N" as `Anexo`,
    -- if(tipo_distribuidor = 'RED', id_distribuidor_linea_de_credito, id_distribuidor) as `Número de documento`,
    id_distribuidor as `Número de documento`,
    date(if(tipo_distribuidor = 'RED',fin_credito,'2099-12-31')) as `Fecha Vencimiento`,
    ROUND(sum(capital), 2) as `Monto Capital`,
    ROUND(sum(interes), 2) as `Monto intérés`,
    ROUND(sum(iva), 2) as `Monto IVA`,
    "FAC" AS `Tipo de Movimiento`
FROM
    (SELECT 
    cd.id_distribuidor,
    "N" as `Anexo`,
    cd.tipo_distribuidor,
    ca.capital,
    ca.interes,
    ca.iva,
    cd.fecha,
    ca.fin_credito,
    ca.no_pagos,
    ca.id_distribuidor_linea_de_credito
    FROM cartera_distribuidor cd
    LEFT JOIN colocacion_asociado ca on cd.id_distribuidor = ca.id_distribuidor
    WHERE fecha = '{fecha_ayer_formato}' and ca.tipo_prestamo = 'PRESTAMO PERSONAL RED') as temp

GROUP BY temp.id_distribuidor, fecha, fin_credito, tipo_distribuidor-- , temp.id_distribuidor_linea_de_credito
            """
cursor.execute(sql_query)
MOVA = cursor.fetchall()

In [199]:
MOVA = pd.DataFrame(MOVA, columns = ['Contrato', 'Anexo', 'Número de documento', 'Fecha Vencimiento',
       'Monto Capital', 'Monto intérés', 'Monto IVA', 'Tipo de Movimiento'])

In [200]:
MOVA

,Contrato,Anexo,Número de documento,Fecha Vencimiento,Monto Capital,Monto intérés,Monto IVA,Tipo de Movimiento
0,81210,N,81210,2021-10-25,34000.00,8245.12,1319.22,FAC
1,81274,N,81274,2021-10-19,54000.00,7373.32,1179.74,FAC
2,81352,N,81352,2021-10-27,33000.00,7867.22,1258.74,FAC
3,81425,N,81425,2021-11-08,32000.00,7630.24,1220.84,FAC
4,81466,N,81466,2021-10-21,35000.00,8341.18,1334.59,FAC
...,...,...,...,...,...,...,...,...
4374,185685,N,185685,2024-08-12,31999.91,8137.72,1302.37,FAC
4375,185721,N,185721,2024-09-09,15242.53,3652.96,584.51,FAC
4376,185725,N,185725,2024-08-12,5338.82,1342.40,214.78,FAC
4377,185754,N,185754,None,4575.16,130.04,20.80,FAC


## Adecuaciones

In [201]:
MOVA['Fecha Vencimiento'] = MOVA['Fecha Vencimiento'].fillna("2099-12-31")
print("Hay que revisar MOVA" if (MOVA.isna().any(0).sum() > 0) else "Todo bien con MOVA")
MOVA['Contrato'] = MOVA['Contrato'].astype(int)
MOVA_id = MOVA['Contrato']
print(MOVA.shape)
MOVA.head(1)

Todo bien con MOVA
(4379, 8)


,Contrato,Anexo,Número de documento,Fecha Vencimiento,Monto Capital,Monto intérés,Monto IVA,Tipo de Movimiento
0,81210,N,81210,2021-10-25,34000.0,8245.12,1319.22,FAC


In [202]:
MOVA = MOVA.loc[[mi in DEU_id for mi in MOVA_id]]
MOVA = MOVA.drop_duplicates('Contrato')
np.setdiff1d(MOVA['Contrato'], DEU_id)

array([], dtype=int32)

In [203]:
new_mova = []
for row in tqdm(MOVA.iterrows()):
    #new_mova = pd.concat([new_mova, row[1]], axis = 1)
    #new_mova.append(row[1])
    num = 16
    for i in range(1, num + 1):
        new_row = row[1].copy()
        new_row['Número de documento'] = str(new_row['Número de documento']) + "_"+str(i)
        new_row['Monto Capital'] = new_row['Monto Capital'] / num
        new_row['Monto intérés'] = new_row['Monto intérés'] / num
        new_row['Monto IVA'] = new_row['Monto IVA'] / num
        #new_mova = pd.concat([new_mova, new_row], axis = 1)
        new_mova.append(new_row)

2427it [00:02, 851.76it/s] 


In [204]:
MOVA = pd.DataFrame(new_mova)
MOVA.head()

,Contrato,Anexo,Número de documento,Fecha Vencimiento,Monto Capital,Monto intérés,Monto IVA,Tipo de Movimiento
0,81210,N,81210_1,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_2,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_3,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_4,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_5,2021-10-25,2125.0,515.32,82.45125,FAC


In [205]:
MOVA.to_csv(f'{path}/{fecha_ayer_formato}/MOVA_{fecha_ayer_formato}.txt', sep ='|', index = False, header = False)

In [206]:
MOVA_ant = pd.read_csv(f"{path}{ultima_fecha}/MOVA_{ultima_fecha}.txt", sep = '|', header = None)
MOVA_ant = MOVA_ant.infer_objects()

MOVA_ant.columns = MOVA.columns
MOVA_ant.shape[0], MOVA.shape[0]

(38816, 38832)

In [207]:
MOVA = pd.read_csv(f"{path}{fecha_ayer_formato}/MOVA_{fecha_ayer_formato}.txt", sep = '|', header = None)
MOVA.columns = MOVA_ant.columns

In [208]:
MOVA_new = encontrar_registros_adicionales(MOVA, MOVA_ant, list(MOVA.columns))

MOVA_new.to_csv(f'{path}/{fecha_ayer_formato}/MOVA_{fecha_ayer_formato}_diferencial.txt', sep ='|', index = False, header = False)

In [209]:
MOVA_ant

,Contrato,Anexo,Número de documento,Fecha Vencimiento,Monto Capital,Monto intérés,Monto IVA,Tipo de Movimiento
0,81210,N,81210_1,2021-10-25,2125.000000,515.32000,82.451250,FAC
1,81210,N,81210_2,2021-10-25,2125.000000,515.32000,82.451250,FAC
2,81210,N,81210_3,2021-10-25,2125.000000,515.32000,82.451250,FAC
3,81210,N,81210_4,2021-10-25,2125.000000,515.32000,82.451250,FAC
4,81210,N,81210_5,2021-10-25,2125.000000,515.32000,82.451250,FAC
...,...,...,...,...,...,...,...,...
38811,185330,N,185330_12,2024-08-09,2312.485625,580.60375,92.910625,FAC
38812,185330,N,185330_13,2024-08-09,2312.485625,580.60375,92.910625,FAC
38813,185330,N,185330_14,2024-08-09,2312.485625,580.60375,92.910625,FAC
38814,185330,N,185330_15,2024-08-09,2312.485625,580.60375,92.910625,FAC


In [210]:
MOVA_new = MOVA[MOVA['Número de documento'].isin(np.setdiff1d(MOVA['Número de documento'], MOVA_ant['Número de documento']))]

MOVA_new = MOVA_new.drop_duplicates(subset = ["Número de documento"])

#Los para enviarse
MOVA_new.to_csv(f'{path}/{fecha_ayer_formato}/MOVA_{fecha_ayer_formato}_tesoreria.csv', index = False)

MOVA_new.to_csv(f'{path}/{fecha_ayer_formato}/MOVA_{fecha_ayer_formato}_diferencial.txt', sep ='|', index = False, header = False)

# MOVIC

In [211]:
cursor = cnx.cursor()
sql_query = f"""
SELECT 
coalesce(cd.id_red, pd.id_distribuidor) as `Contrato`,
"N" AS `Anexo`,
-- coalesce(if(cd.tipo_prestamo = 'PRESTAMO PERSONAL RED', cd.id_red, cd.id_distribuidor), pd.id_distribuidor) as `Número de documento`,
coalesce(cd.id_distribuidor, pd.id_distribuidor) as `Número de documento`,
DATE(pd.fecha_pago) as `Fecha Depósito`,
DATE(pd.fecha_aplicacion_pago) as `Fecha Aplicación`,
IF(pd.seguro_asociado>0,-pd.seguro_asociado,pd.seguro_asociado) as `Monto de Seguro Asociado`,
IF(pd.seguro_distribuidor>0,-pd.seguro_distribuidor,pd.seguro_distribuidor) as `Monto de Seguro Distribuidor`,
"1" AS `Tipo de Movimiento`,
pd.procedencia as `Cuenta`,
pd.id_pago as `paymentId`

FROM pagos_desglosados pd

left JOIN (
		select 
			ca.id_distribuidor, max(ca.id_red) as 'id_red', max(ca.tipo_prestamo) as 'tipo_prestamo'
			from (
				select 
				if(tipo_prestamo = 'PRESTAMO PERSONAL RED', id_distribuidor_linea_de_credito, id_distribuidor) as 'id_distribuidor', 
				id_distribuidor as 'id_red',
				tipo_prestamo
				from colocacion_asociado
			) as ca 
		group by ca.id_distribuidor
) cd on pd.id_distribuidor = cd.id_red
WHERE date(pd.fecha_pago) >='2021-01-01'
"""
cursor.execute(sql_query)
MOVIC = cursor.fetchall()

In [212]:
MOVIC = pd.DataFrame(MOVIC, columns = ['Contrato', 'Anexo', 'Número de documento', 'Fecha Depósito',
       'Fecha Aplicación', 'Monto de Seguro Asociado',
       'Monto de Seguro Distribuidor', 'Tipo de Movimiento', 'Cuenta',
       'paymentId'])

In [213]:
print("Hay que revisar MOVIC" if (MOVIC.isna().any(0).sum() > 0) else "Todo bien con MOVIC")
MOVIC.Contrato = MOVIC.Contrato.astype(int) 

MOVIC_id = MOVIC['Contrato']
print(MOVIC.shape)
MOVIC.head(1)

Todo bien con MOVIC
(327773, 10)


,Contrato,Anexo,Número de documento,Fecha Depósito,Fecha Aplicación,Monto de Seguro Asociado,Monto de Seguro Distribuidor,Tipo de Movimiento,Cuenta,paymentId
0,80782,N,80782,2021-10-04,2021-10-04,-50,-25,1,BANCOMER,105234


In [214]:
sum([mi in DEU_id for mi in MOVIC_id])

254683

In [215]:
MOVIC = MOVIC.loc[[mi in DEU_id for mi in MOVIC_id]]

In [216]:
np.setdiff1d(MOVIC['Contrato'], DEU_id)

array([], dtype=int32)

In [217]:
MOVIC = MOVIC.drop_duplicates(subset = ["paymentId"])

In [218]:
MOVIC.to_csv(f'{path}/{fecha_ayer_formato}/MOVIC_{fecha_ayer_formato}.txt', sep ='|', index = False, header = False)

In [219]:
MOVIC_ant = pd.read_csv(f"{path}{ultima_fecha}/MOVIC_{ultima_fecha}.txt", sep = '|', header = None)
MOVIC_ant = MOVIC_ant.infer_objects()

MOVIC_ant.columns = MOVIC.columns
MOVIC_ant.shape[0], MOVIC.shape[0]

(134008, 134045)

In [220]:
MOVIC = pd.read_csv(f"{path}{fecha_ayer_formato}/MOVIC_{fecha_ayer_formato}.txt", sep = '|', header = None)
MOVIC.columns = MOVIC_ant.columns

In [221]:
MOVIC_new = MOVIC[MOVIC.paymentId.isin(np.setdiff1d(MOVIC['paymentId'], MOVIC_ant['paymentId']))]

MOVIC_new = MOVIC_new.drop_duplicates(subset = ["paymentId"])

#Los para enviarse
MOVIC_new.to_csv(f'{path}/{fecha_ayer_formato}/MOVIC_{fecha_ayer_formato}_tesoreria.csv', index = False)

MOVIC_new.to_csv(f'{path}/{fecha_ayer_formato}/MOVIC_{fecha_ayer_formato}_diferencial.txt', 
                 sep ='|', index = False, header = False)

# MOVO

In [222]:
cursor = cnx.cursor()
sql_query = f"""
select
coalesce(cd.id_red, pd.id_distribuidor) as `Contrato`,
"N" AS `Anexo`,
-- coalesce(if(cd.tipo_prestamo = 'PRESTAMO PERSONAL RED', cd.id_red, cd.id_distribuidor), pd.id_distribuidor) as `Número de documento`,
coalesce(cd.id_distribuidor, pd.id_distribuidor) as `Número de documento`,
DATE(pd.fecha_pago) as `Fecha Depósito`,
DATE(pd.fecha_aplicacion_pago) as `Fecha Aplicación`,
IF(pd.capital>0,-pd.capital,pd.capital) as `Monto de Principal`,
IF(pd.interes>0,-pd.interes,pd.interes) as `Monto de interés`,
IF(pd.iva>0,-pd.iva,pd.iva) as `Monto de IVA`,
"PAG" AS `Tipo de Movimiento`,
pd.procedencia as `Cuenta`,
pd.id_pago as `paymentId`
FROM pagos_desglosados pd
left JOIN (
		select 
			ca.id_distribuidor, max(ca.id_red) as 'id_red', max(ca.tipo_prestamo) as 'tipo_prestamo'
			from (
				select 
				if(tipo_prestamo = 'PRESTAMO PERSONAL RED', id_distribuidor_linea_de_credito, id_distribuidor) as 'id_distribuidor', 
				id_distribuidor as 'id_red',
				tipo_prestamo
				from colocacion_asociado
			) as ca 
		group by ca.id_distribuidor
) cd on pd.id_distribuidor = cd.id_red
where date(fecha_pago) >= '2021-01-01'
            """
cursor.execute(sql_query)
MOVO = cursor.fetchall()

In [223]:
MOVO = pd.DataFrame(MOVO, columns =  ['Contrato', 'Anexo', 'Número de documento', 'Fecha Depósito',
       'Fecha Aplicación', 'Monto de Principal', 'Monto de interés',
       'Monto de IVA', 'Tipo de Movimiento', 'Cuenta', 'paymentId'])

In [224]:
print("Hay que revisar MOVO" if (MOVO.isna().any(0).sum() > 0) else "Todo bien con MOVO")
MOVO['Contrato'] = MOVO['Contrato'].astype(int)
MOVO_id = MOVO['Contrato']
print(MOVO.shape)
MOVO.head(1)

Todo bien con MOVO
(327773, 11)


,Contrato,Anexo,Número de documento,Fecha Depósito,Fecha Aplicación,Monto de Principal,Monto de interés,Monto de IVA,Tipo de Movimiento,Cuenta,paymentId
0,80782,N,80782,2021-10-04,2021-10-04,-1199.75,-103.78,-16.6,PAG,BANCOMER,105234


In [225]:
sum([mi in DEU_id for mi in MOVO_id])

254683

In [226]:
MOVO = MOVO.loc[[mi in DEU_id for mi in MOVO_id]]

In [227]:
np.setdiff1d(MOVO['Contrato'], DEU_id)

array([], dtype=int32)

In [228]:
MOVO = MOVO.drop_duplicates(subset = ["paymentId"])

In [229]:
MOVO.to_csv(f'./{path}/{fecha_ayer_formato}/MOVO_{fecha_ayer_formato}.txt', sep ='|', index = False, header = False)

In [230]:
MOVO_ant = pd.read_csv(f"{path}{ultima_fecha}/MOVO_{ultima_fecha}.txt", sep = '|', header = None)
MOVO_ant = MOVO_ant.infer_objects()

MOVO_ant.columns = MOVO.columns
MOVO_ant.shape[0], MOVO.shape[0]

(134008, 134045)

In [231]:
MOVO = pd.read_csv(f"{path}{fecha_ayer_formato}/MOVO_{fecha_ayer_formato}.txt", sep = '|', header = None)
MOVO.columns = MOVO_ant.columns

In [232]:
MOVO_new = MOVO[MOVO.paymentId.isin(np.setdiff1d(MOVO['paymentId'], MOVO_ant['paymentId']))]

MOVO_new = MOVO_new.drop_duplicates(subset = ["paymentId"])

#Los para enviarse
MOVO_new.to_csv(f'{path}/{fecha_ayer_formato}/MOVO_{fecha_ayer_formato}_tesoreria.csv', index = False)

MOVO_new.to_csv(f'{path}/{fecha_ayer_formato}/MOVO_{fecha_ayer_formato}_diferencial.txt', sep ='|', index = False, header = False)

In [233]:
MOVO_new

,Contrato,Anexo,Número de documento,Fecha Depósito,Fecha Aplicación,Monto de Principal,Monto de interés,Monto de IVA,Tipo de Movimiento,Cuenta,paymentId
133230,177841,N,150324,2024-04-08,2024-04-08,-8417.81,-1813.09,-290.10,PAG,PAYCASH,268948
133709,83730,N,83730,2024-04-15,2024-04-16,-1605.76,-534.69,-85.55,PAG,BANCOMER,270310
133716,181102,N,109210,2024-04-15,2024-04-15,-5761.17,-1336.92,-213.91,PAG,BANCOMER,270002
133723,177279,N,149212,2024-04-15,2024-04-15,-3155.00,-4.34,-0.69,PAG,PAYCASH,270131
133728,180694,N,161991,2024-04-15,2024-04-15,-3358.69,-1203.72,-192.59,PAG,PAYCASH,270143
133729,177685,N,108448,2024-04-15,2024-04-15,-16409.80,-2899.31,-463.89,PAG,BANCOMER,270153
133734,174079,N,121367,2024-04-15,2024-04-15,-2551.19,-99.84,-15.97,PAG,BANCOMER,270223
133736,176899,N,129454,2024-04-15,2024-04-15,-9529.18,-1511.91,-241.91,PAG,PAYCASH,270149
133778,174685,N,101292,2024-04-16,2024-04-17,-867.34,0.00,0.00,PAG,BANCOMER,270620
133790,174685,N,101292,2024-04-16,2024-04-17,-1000.01,0.00,0.00,PAG,BANCOMER,270658


In [234]:
cnx.close()

# Enviando al SFTP

In [235]:
import paramiko

In [236]:
# Configura la conexión SFTP
def enviar(name):
    hostname = 'linqsy.dyndns.org'
    port = 22
    username = 'Volana'
    password = '*v0lana-202311'

    # Crea una instancia de la clase Transport
    transport = paramiko.Transport((hostname, port))

    # Conéctate con el usuario y contraseña
    transport.connect(username=username, password=password)


    # Ruta local y remota del archivo
    local_path = f'./JERS/data/{fecha_ayer_formato}/{name}_{fecha_ayer_formato}_diferencial.txt'
    #remote_path = f'/archivos_{fecha_ayer_formato}/{name}_{fecha_ayer_formato}.txt'
    #
    remote_path = f'/{name}{fecha_ayer_formato.replace("-","")}.txt'
    # Crea una instancia de la clase SFTP
    sftp = paramiko.SFTPClient.from_transport(transport)

    # Sube el archivo
    sftp.put(local_path, remote_path)

    # Cierra la conexión SFTP
    sftp.close()

    # Cierra la conexión de transporte
    transport.close()

In [237]:
for name in ['DEU','MOVO', 'MOVIC', 'MOVA']:
#for name in ['MOVO', 'MOVIC']:
    enviar(name)
    print(f"{name} enviado correctamente")

DEU enviado correctamente
MOVO enviado correctamente
MOVIC enviado correctamente
MOVA enviado correctamente


# Envios por correo

In [238]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from datetime import datetime, timedelta

# Email and password for less secure app access
email = 'jairo.ramirez@volana.com'
password = 'mate22MATE*'

# Recipient email address
to_email = 'neftali.sotelo@volana.com '
Cc_email = ['antonio.herrera@volana.com', 'jairoenrique_ramirez@outlook.com']
# Create a message
msg = MIMEMultipart()
msg['From'] = email
msg['To'] = to_email
msg['Cc'] = ' '.join(Cc_email)
msg['Subject'] = f'Reportes Linq {fecha_ayer_formato}'

body = 'Envio automático de reportes.\nEnrique Ramírez.'
msg.attach(MIMEText(body, 'plain'))

## ======================== Adjuntos ========================

files = ["DEU", "MOVA", "MOVIC", "MOVO"]
#files = ["MOVIC", "MOVO"]
for file in files:

    filename = f'./JERS/data/{fecha_ayer_formato}/{file}_{fecha_ayer_formato}_tesoreria.csv'
    attachment = open(filename, 'rb')

    part = MIMEBase('application', 'octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

    msg.attach(part)

# =================================

# Connect to Gmail's SMTP server
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()

# Login to your Gmail account
server.login(email, password)

# Send the email
server.sendmail(email, [to_email] + Cc_email, msg.as_string())

# Quit the server
server.quit()

print("Email sent successfully!")


Email sent successfully!
